In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import uni2ts
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_next_multi
import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
def get_sample_id(batch_size, seq_len):
    # Creamos un tensor con valores [0, 1, ..., batch_size - 1]
    # Luego lo repetimos seq_len veces a lo largo de la dimensión 1
    return einops.repeat(torch.arange(batch_size), 'b -> b seq', seq=seq_len)
    
def get_variate_id(batch_size, seq_len, n_vars):
    total_repeats = seq_len // n_vars  # Cantidad de veces que cada variate_id debe repetirse para llenar seq_len
    remaining = seq_len % n_vars  # Resto que no completa una división exacta
    var_ids = torch.arange(n_vars).repeat_interleave(total_repeats)
    if remaining > 0:
        var_ids = torch.cat((var_ids, torch.arange(remaining)))
    return var_ids.unsqueeze(0).repeat(batch_size, 1)
    
def get_time_id(batch_size, seq_len, patches_per_var):
    patches_per_var = int (patches_per_var)
    # Creamos un tensor que repite una secuencia de 0 a patches_per_var-1
    time_ids = torch.arange(patches_per_var).repeat(seq_len // patches_per_var + 1)[:seq_len]
    # Repetimos este patrón para cada muestra en el batch
    return time_ids.unsqueeze(0).repeat(batch_size, 1)
def padd_tensor(X, window_len, patch_size, num_variates):
    # Calcular el número total de parches necesarios para una sola variable
    n_patches_single_var = int(np.ceil(window_len / patch_size))
    total_length_needed = n_patches_single_var * patch_size  # Longitud total requerida con padding

    # Calcular el padding necesario si existe
    padding_size = total_length_needed - window_len

    # Añadir padding a X si es necesario
    x_padd = X
    if padding_size > 0:
        x_padd = F.pad(X, (0, padding_size), mode='constant', value=0)  # Pad al final de la última dimensión

    # Ajustar n_patches considerando todas las variables
    n_patches = n_patches_single_var * num_variates
    return x_padd

In [6]:
num_windows, num_variates, window_len = shape = (521, 3, 30) 
X = torch.rand(shape)
batch_size = num_windows
patch_size = 128 #Porque 8 no le gusta y parece que está hecho a 64
max_patch = patch_size 
n_patches = int(np.ceil(window_len/patch_size))*num_variates
x_padded = padd_tensor(X, window_len, patch_size, num_variates)
target = einops.rearrange(
    x_padded,
    'batch_size n_vars (n_patches p) -> batch_size (n_vars n_patches) p',
    p = patch_size
)
batch, seq_len, max_patch = target.shape
observed_mask = torch.ones_like(target, dtype = bool)
prediction_mask = torch.zeros((batch_size, n_patches), dtype = bool)
sample_id = get_sample_id(batch_size, n_patches)
variate_id = get_variate_id(batch_size, n_patches, num_variates)
time_id = get_time_id(batch_size, seq_len, 1)
patch_size_tensor =  torch.zeros((batch_size, n_patches, patch_size))+patch_size

In [7]:
print(f"Target ~ {target.shape}")
print(f"observed_mask ~ {observed_mask.shape}")
print(f"prediction_mask ~ {prediction_mask.shape}")
print(f"sample_id ~ {sample_id.shape}")
print(f"time_id ~ {time_id.shape}")
print(f"variate_id ~ {variate_id.shape}")
print(f"patch_size ~ {patch_size_tensor.shape}")
patch_size_tensor.shape

Target ~ torch.Size([521, 3, 128])
observed_mask ~ torch.Size([521, 3, 128])
prediction_mask ~ torch.Size([521, 3])
sample_id ~ torch.Size([521, 3])
time_id ~ torch.Size([521, 3])
variate_id ~ torch.Size([521, 3])
patch_size ~ torch.Size([521, 3, 128])


torch.Size([521, 3, 128])

In [ ]:
#module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small")

In [16]:
module = MoiraiModule(
    distr_output = None,
    d_model = 32,
    num_layers = 3,
    patch_sizes = [ patch_size ] *batch_size*n_patches*patch_size, 
    max_seq_len = target.shape[2],
    attn_dropout_p = 0.5,
    dropout_p = 0.5,
    scaling = True
)

AssertionError: 

In [ ]:
module(target, observed_mask, sample_id, time_id, variate_id, prediction_mask, patch_size_tensor)